# Solving sudokus with Azure Quantum solvers

Hello! In this Jupyter notebook we will explain and solve Sudoku puzzles with Azure quantum-inspired optimization solvers. This automation step will allow you check your sudoku solutions, or even better, you'll never have to do a soduku manually again! For humans these puzzles can be very challenging and time-consuming, even though the rules of the game are quite simple. An easy use-case to learn more about optimization in an applied way!


### Side Note: 
Here you will learn about how to convert a mixed integer linear programming problem ("MILP") into a quadratic unconstrained binary optimiation ("PUBO") problem. The Azure quantum inspired solvers, or even the specific underlying hardware, **might not be the most suitable for solving** Sudoku problems. Nevertheless, these small scale problems are good learning material for more challenging optimization problems you may face elsewhere. Additionally for very large the sudokus, or optimization solvers in general, heuristic approaches become increasingly attractive in respect to other methods. 


### An example sudoku
Below you can see an example $9 \times9$ sudoku puzzle, which you may use as reference in this notebook. 
- The sudoku is made up of cells, each of which may have only one integer.
- The sudoku has a number of subgrids, "smaller matrices". 


[Sample Sudoku](./example_sudoku.jpg)
- Sample sudoku taken from https://www.andrew.cmu.edu/user/astian/img/example_sudoku.jpg. 


### Interesting notes

- Sudoku is an NP-complete problem ($\text{iff}$ grid size is considered variable $n^2 \times n^2$). In short, this means that finding a solution is difficult (nondeterministic-polynomial time), but verifying the solution can be done easily (polynomial time) and that the problem is reducible to other NP problem variations in polynomial time. 
- Because of the NP-completeness, it is hard to say what kind of solver works 'best'. However, as always, certain algorithms are (or can be) more suitable for a problem. 
    - For a small Sudokus, like $9\times9$, backtracking techniques are useful (like Algorithm X with "Dancing Links"). Backtracking algorithms test out all (partial) solutions to find the one that satisfies the constraints. 
    - For large Sudokus, like $81\times81$, backtracking techniques become computationally expensive. Other methods, such as constraint relaxation to employ linear programming (could also use branch-and-bound) or stochastic optimization (QIO) can then also be considered. These methods have their own specific advantages and disadvantages. In fact, these methods are not even guaranteed to converge to due to the inherent approximations, local minima, non-determinism, etc. 
    - Sudoku puzzles can be considered a graph-coloring problem, each integer-value would be associated with a color. The graph-coloring problem a famous and difficult one, read more about it here: https://en.wikipedia.org/wiki/Graph_coloring. If you're interested in solving the graph coloring problem with Q#, check out this module: https://docs.microsoft.com/en-us/learn/modules/solve-graph-coloring-problems-grovers-search/.   
    - Solving Sudokus can be done with many different solvers, each with their own pros and cons. Here are some links if you want to learn more about heuristics and Microsoft and Partner QIO solvers: 
        - https://glossary.informs.org/ver2/mpgwiki/index.php?title=Algorithm&1=Assignment_problem&2=Heuristic_search.
        - https://docs.microsoft.com/en-us/azure/quantum/qio-target-list.


Sources and more info:
- http://norvig.com/sudoku.html
- https://arxiv.org/abs/1203.2295
- https://en.wikipedia.org/wiki/Sudoku_solving_algorithms
- https://en.wikipedia.org/wiki/Knuth%27s_Algorithm_X (with "Dancing Links")

### Problem background

Sudoku puzzles usually take place on a $9\times9$ grid, but can be any $n^2 \times n^2$ grid, in which many numbers are missing. The goal is to fill in numbers $x_i \in \{1,2,...,n^2\}$ such that the following constraints are satisfied:

- Constraint 1: A number may appear only once per row. 
- Constraint 2: A number may appear only once per column.
- Constraint 3: A number may appear only once per $n\times n$ subgrid.

And that's it! Finding the solution however, which you may assume is unique (depends on the number of blanks), is not straightforward. Solving these puzzles, which belong to a class called "constraint satisfaction problems",  is a lengthy iterative procedure. Fortunately, computers can speed things up a bit! Numerous algorithmic approaches exist, such as backtracking, mixed integer linear programming, simplex method with constraint relaxations, stochastic optimization, or brute force searches. The links in the 'interesting notes' section will refer you to further reading!

For this tutorial, we will consider converting the mixed integer linear programming problem ("MILP"/"ILP") to a binary integer linear programming problem, which will then be cast as a polynomial unconstrained optimization problem ("PUBO"). The difference between these is how the optimization problems are expressed. In MILP, the optimization variables can take any integer value in the set $x_i \in \{1,2,...,n^2\}$. In the binary case, the optimization variables can only take values of 0 and 1, $x_i \in \{0,1\}$. Lastly, for the Azure quantum-inspired solvers to understand the problem, it is necessary to convert the problem to a PUBO (or Ising for other problems) model. 



## Starting point

Throughout this notebook, we will define the problem mathematically and present concepts with pieces of code. Below a snippet is presented that will initialize your connection to Azure, and the function definition in which pieces of code will be appended.

If you don't have an Azure workspace then check out this module:  https://docs.microsoft.com/en-us/learn/modules/get-started-azure-quantum/  

In [ ]:
#import dependencies 
import numpy as np
import math
from collections import defaultdict
from azure.identity import ClientSecretCredential
from azure.quantum import Workspace
from azure.quantum.optimization import Problem, ProblemType, Term
from azure.quantum.optimization import SimulatedAnnealing, ParallelTempering, Tabu, QuantumMonteCarlo, HardwarePlatform, Solver


workspace = Workspace (
    subscription_id = "",
    resource_group = "",
    name = "",
    location = ""
)



##### Some example sudokus. '0's represent blank cells. 

Sudoku4A = np.matrix([[0, 0, 1, 0],
                      [0, 0, 2, 0],
                      [0, 2, 0, 0],
                      [0, 4, 0, 0]
                    ])


Sudoku4B = np.matrix([[0, 0, 0, 0],
                      [1, 0, 3, 0],
                      [4, 3, 1, 0],
                      [2, 0, 0, 0]
                    ])


Sudoku9A = np.matrix([[1, 0, 0, 0, 5, 0, 0, 0, 3], 
                      [0, 0, 0, 0, 2, 1, 9, 0, 4],
                      [5, 9, 0, 0, 0, 6, 0, 0, 0],
                      [0, 3, 0, 0, 4, 0, 6, 1, 0],
                      [0, 0, 7, 0, 0, 3, 8, 0, 0],
                      [4, 0, 0, 1, 7, 0, 0, 0, 0],
                      [7, 0, 0, 9, 0, 5, 0, 3, 0],
                      [9, 8, 5, 0, 0, 0, 2, 0, 0],
                      [0, 0, 0, 0, 6, 0, 0, 0, 8]
                      ])


Sudoku9B = np.matrix([[0, 0, 0, 6, 0, 0, 4, 0, 5],
                      [0, 8, 6, 0, 0, 9, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 8, 7, 0],
                      [6, 2, 0, 7, 0, 0, 3, 8, 0],
                      [0, 0, 0, 0, 0, 1, 0, 0, 9],
                      [5, 0, 3, 0, 0, 0, 0, 0, 4],
                      [0, 4, 7, 0, 9, 2, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 1, 2],
                      [8, 0, 0, 0, 3, 0, 5, 0, 0]
                     ])

############################################################################################
##### Generate problem instance
def SudokuProblem(SudokuMatrix):

    terms = []
    N = len(SudokuMatrix)

    #####
    # Code snippets will be appended here!
    #####

    return Problem(name="SudokuOptProblem", problem_type=ProblemType.pubo, terms=terms)




############################################################################################
##### Generate cost function
OptimizationProblem = SudokuProblem(Sudoku9A)



## The MILP formulation

Here, a mixed integer linear programming model is introduced to solve Sudokus. For simplicity, the variables to be filled into the soduku $x$ are indexed according to the rows ($r$) and columns ($c$) is taken. The problem formulation expressed mathematically:   

$$ \underset{\small{x_{r,c} \in { \{1,...,n^2}\}}}{min} \hspace{0.1cm} \hspace{0.25cm} 0 $$

$$ \text{subject to: } $$
$$ \sum_{r=0}^{n^2-1}\sum_{c=0}^{n^2-1}  x_{r,c} = \sum_{v=1}^{n^2} v \hspace{0.1cm},\hspace{0.3cm} \text{the sum of the row cells is the sum of the values in the set } \{1,2,...n^2\} \hspace{0.1cm} \text{(constraint 1)}.$$ 
$$ \sum_{c=0}^{n^2-1}\sum_{r=0}^{n^2-1}  x_{r,c} = \sum_{v=1}^{n^2} v \hspace{0.1cm},\hspace{0.3cm} \text{the sum of the column cells is the sum of the values in the set } \{1,2,...n^2\}  \hspace{0.1cm}\text{(constraint 2)}.$$
$$ \sum_{\alpha=0}^{n-1} \sum_{\beta=0}^{n-1} \hspace{0.2cm} \sum_{r = n\alpha+1}^{n(\alpha+1)} \hspace{0.2cm} \sum_{r = n\beta+1}^{n(\beta+1)} x_{r,c}   = \sum_{v=1}^{n^2} v  \hspace{0.1cm},\hspace{0.3cm}  \text{the sum of the cells in a subgrid is the sum of the values in the set } \{1,2,...n^{2} \}  \hspace{0.1cm} \text{(constraint 3)}.$$ 
$$ $$

These are the constraints that were covered in the problem background. As you can see there are only constraints, no objective function. The task is to find the unique solution such that all constraints are satisfied.  

> There are sudokus in which multiple solutions exist, you can solve those with the code in this notebook as well. But usually only have 1 solution however. If there are multiple solutions due to too many initial blanks, then there is no way of measuring the 'best' solution with regards to the constraints. As a consequence there can be no global minima in such a scenario, only local minima with the same value (cost function value at the minima). 

Because all numbers in a row, column, or subgrid must be different (unique) but inside $\{1,2,...n^2\}$, it should clear that each of these sums should all equal all elements of the set added together. That is how these constraints are modeled. The first constraint is the sum a rows' cells, achieved by iterating over a row's column indices. Similarly, the second constraint describes the sum of the columns' cells, adding the elements over a column's row indices. Lastly, the sum of all cell elements in each subgrid (indexed by $(\alpha, \beta)$) is calculated. 



## Converting to a binary optimization problem - variables

For the Azure solvers it is necessary to recast the sudoku optimization problem to a binary form. There are two available formats, the Ising model $x_i \in \{-1,1\}$ and the PUBO/QUBO model $x_i \in \{0,1\}$. Here we will consider the the second, which is the easiest way to rewrite the problem. 

Going from $x_i \in \{1,2,...n^2\}$ to $x_i \in {0,1}$ requires a different formulation for the problem. Consider a single cell in which we want to assign a particular value. Now it is not possible to simply assign a number in the set. However what we can do is abstract a little bit. Representing each value in the set $\{1,2,...n^2\}$ by a binary variable $x_v$ is neat trick to convert to a binary problem. For example:

$$ \text{The value 1 can be represented by } x_{{1}}$$
$$ \text{The value 2 can be represented by } x_{{2}}$$
$$ \text{The value v can be represented by } x_{{v}}$$

Now if a value 2 needs to be filled into a cell, we can describe that by assigning $x_2=1$, and keeping all other variables equal to zero. Why do other variables need to be zero? The reason is that we are **selecting per cell**, and only one value may be filled in a cell. Therefore there are $n^2$ variables per cell in order to describe which integer needs to be filled in. 

To eloborate on the cell values, the row and column number needs to be given to distinguish integers assigned to different cells. For simplicity, we can assume an indexing of $x_{{r,c,v}}$, with $r$ denoting the row number, $c$ the column number, and $v$ representing the value in the set integer set. For example, if we want to assign cell (8,8) a value in the shown picture above, which is the most bottom right cell (cell count starts at 0), then we'd have to choose from the set of variables:


$$ \{ x_{{8,8,1}}, x_{{8,8,2}},...,x_{{8,8,n^2}}   \} \text{ representing values: } \{1,2,...,n^2\}.$$


More generally, recasting to the binary format gives the following points: 
- per cell there are $n^2$ variables, each represent a different value (v): $x_{r,c,v}$ 
- each variable is associated (indexed) with a specific cell according to the row(r) and column(c): $x_{r,c,v}$
- there are $n^2$ values per cell, and $n^2$ cells per row/column. Therefore there are $n^6$ different binary variables ($x_{r,c,v}$) in total. 


**NOTE**: The solver does not understand the format $x_{r,c,v}$ which is why a slightly different notation will be used from here on: $x_k = x_{ r n^4 + c n^2 + v}$


Very nice. With the recast variables we can start looking at the optimization problem itself!






## One value per cell constraint

Because of the reformulation, there are $n^2$ variables per cell. Only one of these variables may be assigned a value 1 by the solver, as the rules of Sudoku prohibit filling in multiple integers per cell. For this, another constraint has to be added to the problem to avoid multiple variables per cell becoming nonzero. It is necessary to penalize relationships between variables that reside in the same cells. One easy way to accomplish this is to penalize the products of variables. Considering a single cell, you could model the constraint as: 

$$ x_v \text{  with } v \in \{1,2,...n^2\}, \hspace{.5cm}\text{(variables for one cell)}$$
$$ x_1 \cdot x_2 \dots x_{n^2} = 0.$$

However, if only one of these $x_v$ takes value 0 and the rest would equal zero the constraint is satisfied, meaning that this model is useless. Investigating the constraint more thoroughly, you may realize that the constituent parts must also be zero. The constraint of the $n^2$ variables can be split. Only one variable may be assigned a value 1 by the solver, essentially meaning that we have to check each dual combination between variables. If all are zero, then you can be sure that **no more** than one variable has value 1. Mathematically the idea is expressed as:

$$ x_v \text{  with } v \in \{1,2,...n^2\}, \hspace{.5cm}\text{(variables for one cell)}$$
$$ x_1 \cdot x_2 = 0,$$
$$ x_1 \cdot x_3 = 0,$$
$$ \vdots  $$ 
$$ x_1 \cdot x_{n^2} = 0,$$
$$ x_2 \cdot x_3 = 0,$$
$$ x_2 \cdot x_4 = 0,$$
$$ \vdots $$ 

Reverse combinations are not considered because that leads to an inbalanced cost function. It is not required to prevent double weighted terms caused by reverse combinations, however the tuning process can become more difficult.  


With some fine tuning later on (contraint tuning), this model should prevent multiple integers being assigned to a single cell. Written out as a constraint function we derive:

$$ \sum_{r = 0}^{n^2-1} \sum_{c = 0}^{n^2-1} \sum_{ref = 0}^{n^2-1} \sum_{tar = 0}^{n^2-1} x_{(rn^4 + n^2c+ref)} \cdot x_{(n^4r+n^2c+tar)}= 0 \hspace{0.5cm } \text{ with: \{ref<tar\} }$$

The formula looks complicated, but it is actually much simpler than it looks. The first two summations select a cell (row, column). The third summation specifies a reference variable in that cell. The fourth summation selects a target variable, in order to weight the reference-target variable combination. The reason "ref" needs to be smaller than "tar" is to avoid reverse combinations, which could also be integrated mathematically into the summation if you would like.


Let's write some code:



In [ ]:

############################################################################################
##### Generate problem instance
def SudokuProblem(SudokuMatrix):

    terms = []
    n = int(np.sqrt(len(SudokuMatrix)))

    #####
    ##### Constraint 1: Per cell, only one variable may be '1', the others must be zero => then only one integer is assigned to the cell

    for r in range(0, pow(n,2)):                                    # iterate over the rows of the matrix
        for c in range(0,pow(n,2)):                                 # iterate over the columns of the matrix
            for ref in range(0,pow(n,2)):                           # select the reference variable in cell
                for tar in range(0,pow(n,2)):                       # select the target variable in cell  
                    if ref<tar:                                     # prevent weighting combinations twice, therefore ref<tar
                        terms.append(
                            Term(
                                c = 1,
                                indices = [(pow(n,4)*r + pow(n,2)*c+ref),(pow(n,4)*r + pow(n,2)*c+tar)]   
                            )
                        )
                        # uncomment if you want to see the weighting combinations
                        #print(f'{(pow(n,4)*r + pow(n,2)*c+ref)},{(pow(n,4)*r + pow(n,2)*c+tar)}')   

    return Problem(name="SudokuOptProblem", problem_type=ProblemType.pubo, terms=terms)



############################################################################################
##### Generate cost function
OptimizationProblem = SudokuProblem(Sudoku4A)

## Sum of the rows' cells

Each row sum must equal the sum of the unique integers that can be filled in. Because each integer in a row must  be unique, we will be weighting combinations of variables that represent the same integer. This will force each row to consist of only unique integers, and hence will automatically ensure that the summation of the cells equal the sum of unique integers. Consider a general $9 \times 9$ sudoku, $x_3$ and $x_{12}$ both represent the integer '4' (first row, first and second cell), the solver may not set both equal to 1. To avoid that from happening it is necessary to penalize this relation, as is done by:

$$x_3 \cdot x_{12} = 0 $$

This penalizes the solver for filling in a '4' in the first and second cell. Of course, the penalazation needs to be expanded all integers and for all the rows' cells. Each variable associated with a specific integer are separated by factor $n^2$. For example, take the first row and the variables representing the integer '4': 

$$x_3 \text{ in first cell and } x_{12} \text{ in the second cell are separated by } n^2=9; \text{   } x_3 \text{ in the first cell and } x_{21} \text{in the third cell are separated by } 2n^2 = 18. $$

Generalizing this to all rows gives the following constraint formula:

$$ \sum_{r = 0}^{n^2-1} \hspace{0.2cm} \sum_{ref=r n^4}^{ (r+1) n^4 - 1 } \hspace{0.2cm} \sum_{tar = ref, \text{ stepsize: } n^2}^{(r+1)n^4 - 1} x_{ref} \cdot x_{tar} = 0 \text{ with ref<tar }$$

The first summation selects the row of a Sudoku. The second summation selects the variable for a reference integer. The third summation selects a target variable, which represents the same integer but has a larger index value ($r \cdot c \cdot v$). The last summation starts at "tar" such that the difference between "ref" and "tar" is exactly $n^2$. Also, the condition "ref"<"tar" is because we don't want to weight the reference variable with itself ("ref"-"ref" combination).

In [ ]:

############################################################################################
##### Generate problem instance
def SudokuProblem(SudokuMatrix):

    terms = []
    n = int(np.sqrt(len(SudokuMatrix)))

    #####
    ##### Constraint 1: Per cell, only one variable may be '1', the others must be zero => then only one integer is assigned to the cell

    for r in range(0, pow(n,2)):                                    # iterate over the rows of the matrix
        for c in range(0,pow(n,2)):                                 # iterate over the columns of the matrix
            for ref in range(0,pow(n,2)):                           # select the reference variable in cell
                for tar in range(0,pow(n,2)):                       # select the target variable in cell  
                    if ref<tar:                                     # prevent weighting combinations twice, therefore ref<tar
                        terms.append(
                            Term(
                                c = 1,
                                indices = [(pow(n,4)*r + pow(n,2)*c+ref),(pow(n,4)*r + pow(n,2)*c+tar)]   
                            )
                        )
                        # uncomment if you want to see the weighting combinations
                        #print(f'{(pow(n,4)*r + pow(n,2)*c+ref)},{(pow(n,4)*r + pow(n,2)*c+tar)}')   
    
    ####################################################################################################################################
    ##### Constraint 2: Per row, an integer may only appear once! 
    for r in range(0,pow(n,2)):                                # iterate over the rows of the matrix                                     
        for ref in range(r*pow(n,4),(r+1)*pow(n,4)):           # select reference variable     
            for tar in range(ref,(r+1)*pow(n,4),pow(n,2)):     # select a target variable, which is the reference variable plus a multiple of N (same int but next cell)
                if ref<tar:                                    # prevent weighting combinations twice, therefore ref<tar
                    terms.append(
                        Term(
                            c = 1,
                            indices = [ref,tar]   
                        )
                    )
                    ##### uncomment if you want to see the weighting combinations
                    #print(f'{ref},{tar}')

    return Problem(name="SudokuOptProblem", problem_type=ProblemType.pubo, terms=terms)



############################################################################################
##### Generate cost function
OptimizationProblem = SudokuProblem(Sudoku4A)


## Sum of the columns' cells

Similarly to the previous constraint, the sum of the cells in each column must equal the sum of the different integers. Luckily, this constraint is easier to formulate. We need to pick reference variables from the first row's cells, and then compare them to their respective counterparts (for the associated integer) in the cells below. The steps between the variables is a factor always a factor $n^4 = 81$. For example, for the first column $x_3$ is selected as a refence variable, denoting the integer '4'. For the cell below (second row, first column) the variable representing '4' is $x_{84}$. For the constraint we then weight the combination of these two to avoid both being assigned a value 1 by solver:

$$ x_{3} \cdot x_{84} = 0 $$ 

The same has to be done for all same-integer combinations:

$$x_{3} \cdot x_{165} $$
$$ \vdots $$
$$x_{84} \cdot x_{165}$$
$$\vdots$$

This constraint can be written as following:

$$ \sum_{ref=0}^{n^6-1} \hspace{0.3cm }\sum_{tar=ref, \text{ stepsize: } n^4 }^{n^6-1} x_{ref} \cdot x_{tar} = 0 \text{ with ref<tar }$$

The first summation selects a reference variable. The second summation selects a target variable associated with the same integer, and starts from "tar" = "ref" such that the difference between variables' indices is exactly $n^4$. As before, the "ref" < "tar" makes sure that the reference variable is not weighted against itself.  


In [ ]:
      
############################################################################################
##### Generate problem instance
def SudokuProblem(SudokuMatrix):

    terms = []
    n = int(np.sqrt(len(SudokuMatrix)))

    #####
    ##### Constraint 1: Per cell, only one variable may be '1', the others must be zero => then only one integer is assigned to the cell

    for r in range(0, pow(n,2)):                    # iterate over the rows of the matrix
        for c in range(0,pow(n,2)):                 # iterate over the columns of the matrix
            for ref in range(0,pow(n,2)):           # select the reference variable in cell
                for tar in range(0,pow(n,2)):       # select the target variable in cell  
                    if ref<tar:                     # prevent weighting combinations twice, therefore ref<tar
                        terms.append(
                            Term(
                                c = 1,
                                indices = [(pow(n,4)*r + pow(n,2)*c+ref),(pow(n,4)*r + pow(n,2)*c+tar)]   
                            )
                        )
                        # uncomment if you want to see the weighting combinations
                        #print(f'{(pow(n,4)*r + pow(n,2)*c+ref)},{(pow(n,4)*r + pow(n,2)*c+tar)}')   
    
    #####
    ##### Constraint 2: Per row, an integer may only appear once! 
    for r in range(0,pow(n,2)):                                # iterate over the rows of the matrix                                     
        for ref in range(r*pow(n,4),(r+1)*pow(n,4)):           # select reference variable     
            for tar in range(ref,(r+1)*pow(n,4),pow(n,2)):     # select a target variable, which is the reference variable plus a multiple of N (same int but next cell)
                if ref<tar:                                    # prevent weighting combinations twice, therefore ref<tar
                    terms.append(
                        Term(
                            c = 1,
                            indices = [ref,tar]   
                        )
                    )
                    ##### uncomment if you want to see the weighting combinations
                    #print(f'{ref},{tar}')
    
    #####
    ##### Constraint 3: Per column, an integer may only appear once! 
    
    for ref in range(0,pow(n,6)):                            # select reference variable 
        for tar in range(ref,pow(n,6),pow(n,4)):             # select target variable, which is the reference variable plus a multiple of N^2 (same int but next row)
            if ref<tar:                                      # prevent weighting combinations twice, therefore ref<tar
                terms.append(
                    Term(
                        c = 1,
                        indices = [ref,tar]   
                    )
                )
                ##### uncomment if you want to see the weighting combinations
                #print(f'{ref},{tar}')

    return Problem(name="SudokuOptProblem", problem_type=ProblemType.pubo, terms=terms)



############################################################################################
##### Generate cost function
OptimizationProblem = SudokuProblem(Sudoku9A)
    
    

## Sum of the subgrid's cells

The sum of each subgrid's cells must equal the sum of the different integers. Modeling this constraint is a bit tricky because we have to iterate over variables that are not separated by a constant value, as in the previous constraints. For ease, let's split the problem into two subproblems. In the first, we'll go over finding the variables in a subgrid and store them in a dictionary. In the second, we'll use the dictionary to weight variable combinations to form the constraint. 

### 1. Finding the variables in a subgrid

A sudoku consists of $n^2$ subgrids ($n \times n$), each of which can be understood as a $n \times n$ matrix. The idea is that we will first create an indexing for these subgrids. For example in a $9 \times 9$ Sudoku "$A$", $A(0,0)$ will denote the top-left subgrid and $A(2,2)$ the bottom-right subgrid. This will allow us to easily specify variables later on. Realizing that we will need to iterate over all subgrids gives us the first part summations, two 'for' loops to specicy each subgrid. 

- 'sg_r' stands for "subgrid_row", of which there are $n$.
- 'sg_c' stands for "subgrid_column", of which there are $n$. 


In [ ]:
##### 1. First get the variables for each sub-grid (sg)  and store it in a dictionary
n = int(np.sqrt(len(Sudoku9A)))            # specify one of the sudokus, as this will be added to the function later (n is defined in the function)

sg_dict = defaultdict(dict)                # define a dict for storing the variables per subgrids
for sg_r in range(0,n):                    # iterate over of row indices for the subgrids
    for sg_c in range(0,n):                # iterate over of column indices for the subgrids
        sg_dict[sg_r][sg_c] = []           # intialize list for a new subgrid
        

Now we have to find all the integers' index values for each cell in the subgrid. First we specify the row of the subgrid we should iterate over (there are $n$ rows in a subgrid), starting at the top row and ending at the bottom row of the subgrid. For each subgrid row, we have to iterate over the number of variables, of which there are $n^3$. This results in two more 'for' loops:

- 'row_num' stands for the row in the subgrid (top to bottom), of which there are $n$.
- 'i' is an iterable for the number of variables per row in a subgrid. There are $n^3$ variables per row in a subgrid .




In [ ]:
##### 1. First get the variables for each subgrid (sg)  and store it in a dictionary
n = int(np.sqrt(len(Sudoku9A)))                 # specify one of the sudokus

sg_dict = defaultdict(dict)                         # define a dict for storing the variables per subgrid
for sg_r in range(0,n):                             # iterate over of row indices for the subgrids
    for sg_c in range(0,n):                         # iterate over of column indices for the subgrids
        sg_dict[sg_r][sg_c] = []                    # intialize list for a new subgrid 
        for row_num in range(0,n):                  # iterate over rows of a subgrid (is multiplied by n^4 later)
            for i in range(0,pow(n,3)):             # iterate over number of variables per row in a subgrid   
                print('some math expressions') # print something nice                      

Great, now that we know the 'for' loop structure, we need an equation to calculate the index values for the integers. 
- For iterating over the variables in a cell, we can use 'i'.
- For iterating over the rows in a subgrid, we know that each subgrid row has $n^4$ variables.
- For iterating over the sudokus subgrid indices, we know that a subgrid row index contributes $n^5$ variables.
- For iterating over the sudokus subgrid indices, we know that a subgrid column index contributes $n^3$ variables.

This leads to the following expression for deriving a subgrid's variables in a list:


In [ ]:
##### 1. First get the variables for each sub-grid (sg)  and store it in a dictionary
n = int(np.sqrt(len(Sudoku9A)))       # specify one of the sudokus

sg_dict = defaultdict(dict)                                                             # define a dict for storing the variables per box
for sg_r in range(0,n):                                                                 # iterate over of row indices for the boxes
    for sg_c in range(0,n):                                                             # iterate over of column indices for the boxes
        sg_dict[sg_r][sg_c] = []                                                        # intialize list for a new box 
        for row_num in range(0,n):                                                      # iterate over rows of a box (is multiplied by N^2 later)
            for i in range(0,pow(n,3)):                                                 # iterate over number of variables per row in a box                                                          
                variable = int(i+(row_num*pow(n,4))+(sg_r*pow(n,5)+sg_c*pow(n,3)))      # compute variable number and append to list
                sg_dict[sg_r][sg_c].append(variable)                                    # append variable list for that box inside a dict                 
                ##### uncomment if you want to see the variables
                #print(f'Added {variable} in list of box({sg_r},{sg_c})')

Awesome! Now that you have the subgrid's variables, we can start weighting integer combinations. This is to avoid integers from occuring multiple times inside a single subgrid. However we don't need to weight combinations between variables on the same rows and columns, as that was already done in the previous constraints. 

As we've constructed a dictionary of lists of all subgrid variables, it has become much easier to start making combinations. The main reason being that the integer variables in the lists are already sorted according to the subgrid indices in a list. The following terms are introduced to make life easier:

- we use 'r' to denote the reference variable in a list
- we use 't' to denote the target variable in a
- we use the 'shift' to select a new starting ('r') integer variable in the list

The assignment has become to iterate over a list (very common problem), for which respective variable combinations representing the same integer need to be weighted. The above variables select members of the list as in the visually decribed below:


![example](./subgrid_list_iter.png)

In order to select the same integer variables, we need to make sure the distance between the list terms is $n^2$. By moving right to left in the list, we make sure all combinations are encountered at least once for an integer. This can be seen in the 'for' loops.

But now the big question remains, how do we weight the combinations of variables that are not on the same row or column? Those are the only combinations that need to be assigned weights, right?

Correct, so what we do is calculate the row and column for each variable (row and column for the entire Sudoku). This is to make sure the integers are not horizontally or vertically aligned, because this has already been considered in previous constraints. If they are not aligned, we can weight the combinations, since it means they are inside the same subgrid but not on the same row or column. This is described by the terms:

- row_num_r  - row for the reference 
- col_num_r  - column for the reference
- row_num_t  - row for the target
- col_num_t  - column for the target






In [ ]:
##### 2. Now weight the variables that represent the same value and are inside the same box 

terms = []

for sg_r in range(0,n):                                     # select subgrid row                                                                               
    for sg_c in range(0,n):                                 # select subgrid column
        sg = sg_dict[sg_r][sg_c]                            
        for shift in range(0,pow(n,2)):                     # select shift as in explanation    
            for r in range(shift,pow(n,4),pow(n,2)):        # select 'r' (reference variable) as in explanation
                for t in range(r,pow(n,4),pow(n,2)):        # select 't' (target variable) as in explanation
                        
                    row_num_r = math.floor(sg[r]/pow(n,4))                  # calculate the row number for the reference variable
                    col_num_r = math.floor((sg[r]%pow(n,4))/pow(n,2))       # calculate the column number for the reference variable
                    row_num_t = math.floor(sg[t]/pow(n,4))                  # calculate the row number for the target variable
                    col_num_t = math.floor((sg[t]%pow(n,4))/pow(n,2))       # calculate the column number for the target variable

                    #print(f'ref:{row_num_r},{col_num_r}')
                    #print(f'tar:{row_num_t},{col_num_t}')
                    
                    if row_num_r != row_num_t and col_num_r != col_num_t:
                        
                        terms.append(
                           Term(
                               c = 1,
                               indices = [sg[r],sg[t]]   
                           )
                        )
                    ### Uncomment to see the variable combinations
                    #print(f'{sg[r]},{sg[t]}') 

A mathematical formulation of this constraint is a bit messy, therefore I've not presented it. If you're interested, you can derive it from the 'for' loops in the function!

## Promote the solver to fill in values

Now we have to promote the solver to fill in values. If this is not done, then the sudoku will not be filled in, therefore we have to encourage the solver. This is done by assigning nearly each variable (not a combination!) a negative weight. This will encourage the solver to assign a variable ($x$) a '1' value. We do need to take into account the initial values that are filled already, these must be given a much more negative weight to discourage the solver from changing them. To implement this constraint, we simply iterate over the variables of the Sudoku.

Below you can see how that this is done.

In [ ]:
#####
##### Constraint 5: Promote to fill in values - Values already in the sudoku get a higher weighting! Can't use Term outside of function def

# To run, uncomment the next two lines
SudokuMatrix = Sudoku9A
terms = []

for row in range(0,pow(n,2)):                        # iterate over rows
    for col in range(0,pow(n,2)):                    # iterate over columns
        for val in range(1,pow(n,2)+1):              # iterate over possible integer values
            if SudokuMatrix[row,col] == val:         # if there is a value filled in, give a much lower/more negative weight
                terms.append(                                                  
                    Term(
                        c = -5,
                        indices = [(val-1)+col*pow(n,2)+row*pow(n,4)]   
                    )
                )
                #print('Directly influenced by initial conditions')
                #print(f'{(val-1)+col*pow(n,2)+row*pow(n,4)}')
            elif SudokuMatrix[row,col] == 0:         
                # if no value is filled in that cell, promote the solver to fill in a weight
                terms.append(
                    Term(
                        c = -1,
                        indices = [(val-1)+col*pow(n,2)+row*pow(n,4)]   
                    )
                )

                #print('Not directly influenced by initial conditions')
                #print(f'{(val-1)+col*pow(n,2)+row*pow(n,4)}') 


## Verification of solutions

Great! That was the last constraint. As you may have noticed, I've already defined the constraint weights in the covered function definitions. This should help you find suitable solutions when running the sample code. But feel free to adjut their values once you are up to the task!

The optimization problem is ready to be submitted to the Azure QIO solvers. However, because it is possible that the solvers return incorrect solutions, based on the values you pass through for the solver parameters and constraint weight, the solutions will need to be verified. 
In the verification, all the constraints are checked to make sure there are no violations. If no violations are detected, then we can be sure the sudoku was solved correctly!

Below you can find the code for verifying the solutions.

In [ ]:
def ReadResults(Config: dict, SudokuMatrix):

    N = len(SudokuMatrix)

    #############################################################################################
    ##### Read the solver's solution (dictionary) and sort/order it 
    SortedAns = {}
    print('\n')
    SortedKeys = sorted({int(k):v for k,v in Config.items()}.keys())
    SortedAns = {k:Config[str(k)] for k in SortedKeys}
    #print(SortedAns)

    #############################################################################################
    ##### Iterate over solution and fill in the matrix
    SolvedSudoku = SudokuMatrix

    for k,v in SortedAns.items():
        if v==1:
            row_num = math.floor(k/pow(N,2))
            col_num = math.floor((k%pow(N,2))/N)                        
            val_num = math.floor(((k%pow(N,2))%N))+1
            SolvedSudoku[row_num,col_num] = val_num


    print(SolvedSudoku)

    InvalidSolution = VerifyResults(SortedAns, SolvedSudoku)


    return InvalidSolution



def VerifyResults(SortedAns, SolvedSudoku):

    N = len(SolvedSudoku)
    InvalidSolution = False

    ####################################################################################################################################
    ##### Check whether any cell has value 0
    for row in range(0,N):
        for col in range(0,N):
            if SolvedSudoku[row,col] == 0:
                #print('A zero was found - invalid')
                #print(f'{row},{col}')
                InvalidSolution = True

    ####################################################################################################################################
    ##### Check constraint 1: Per cell, only one variable may be '1', the others must be zero => then only one integer is assigned to the cell   
    for cell in range(0,pow(N,3),N):
        cell_sum = 0
        #print('\n')
        for i in range(0,N):
            #print(cell+i)
            cell_sum += SortedAns[cell+i]
        if cell_sum > 1:
            InvalidSolution = True
            print('More than one value per cell')
            print(SortedAns[cell+i])
            #raise RuntimeError(f'Too many cell values! {cell_sum} were specified for box ({math.floor((cell+i)/pow(N,2))},{math.floor(((cell+i)%pow(N,2))/N)})' )
        elif cell_sum == 0:
            print('Less than one value per cell')
            print(SortedAns[cell+i])
            InvalidSolution = True
            #raise RuntimeError(f'Too few cell values! {cell_sum} were specified for box ({math.floor((cell+i)/pow(N,2))},{math.floor(((cell+i)%pow(N,2))/N)})' )

    ####################################################################################################################################
    ##### Constraint 2: Per row, an integer may only appear once! 

    for i in range(0,N):
        for j in range(0,N):
            for k in range(0,N):
                if k>j:
                    if SolvedSudoku[i,k] == SolvedSudoku[i,j]:
                        #print(f'{i},{j},{k}')
                        print(f'Duplicate int in row {i+1}')
                        InvalidSolution = True

    ####################################################################################################################################
    ##### Constraint 2: Per column, an integer may only appear once! 

    for j in range(0,N):
        for i in range(0,N):
            for k in range(0,N):
                if k>i:
                    if SolvedSudoku[k,j] == SolvedSudoku[i,j]:
                        #print(f'{j},{i},{k}')
                        print(f'Duplicate int in column {j+1}')
                        InvalidSolution = True

    print(f"Is this solution valid? {not InvalidSolution}")

    return InvalidSolution

Below the function definition to submit to the Azure solvers is presented, in which you can choose a solver to your liking (there are many more available, https://docs.microsoft.com/en-us/azure/quantum/qio-target-list). The execution time for these problems should be a few seconds at most. You can try improving the cost function and solver paramters to decrease that time. Do try out some parameterized solvers to get some experience tuning the solvers!

In [ ]:
############################################################################################
##### Generate problem instance
def SudokuProblem(SudokuMatrix):

    terms = []
    n = int(np.sqrt(len(SudokuMatrix)))

    #####
    ##### Constraint 1: Per cell, only one variable may be '1', the others must be zero => then only one integer is assigned to the cell

    for r in range(0, pow(n,2)):                   # iterate over the rows of the matrix
        for c in range(0,pow(n,2)):                # iterate over the columns of the matrix
            for ref in range(0,pow(n,2)):          # select the reference variable in cell
                for tar in range(0,pow(n,2)):      # select the target variable in cell  
                    if ref<tar:                    # prevent weighting combinations twice, therefore ref<tar
                        terms.append(
                            Term(
                                c = 1,
                                indices = [(pow(n,4)*r + pow(n,2)*c+ref),(pow(n,4)*r + pow(n,2)*c+tar)]   
                            )
                        )
                        # uncomment if you want to see the weighting combinations
                        #print(f'{(pow(n,4)*r + pow(n,2)*c+ref)},{(pow(n,4)*r + pow(n,2)*c+tar)}')   

    #####
    ##### Constraint 2: Per row, an integer may only appear once! 
    for r in range(0,pow(n,2)):                                # iterate over the rows of the matrix                                     
        for ref in range(r*pow(n,4),(r+1)*pow(n,4)):           # select reference variable     
            for tar in range(ref,(r+1)*pow(n,4),pow(n,2)):     # select a target variable, which is the reference variable plus a multiple of N (same int but next cell)
                if ref<tar:                                    # prevent weighting combinations twice, therefore ref<tar
                    terms.append(
                        Term(
                            c = 1,
                            indices = [ref,tar]   
                        )
                    )
                    ##### uncomment if you want to see the weighting combinations
                    #print(f'{ref},{tar}')
    
    #####
    ##### Constraint 3: Per column, an integer may only appear once! 
    
    for ref in range(0,pow(n,6)):                    # select reference variable 
        for tar in range(ref,pow(n,6),pow(n,4)):     # select target variable, which is the reference variable plus a multiple of N^2 (same int but next row)
            if ref<tar:                              # prevent weighting combinations twice, therefore ref<tar
                terms.append(
                    Term(
                        c = 1,
                        indices = [ref,tar]   
                    )
                )
                ##### uncomment if you want to see the weighting combinations
                #print(f'{ref},{tar}')
            
    #####
    ##### Constraint 4: In each box/area, an integer may only appear once!
   
    ##### 1. First get the variables for each sub-grid (sg)  and store it in a dictionary
    n = int(np.sqrt(len(Sudoku9A)))       # specify one of the sudokus
    
    sg_dict = defaultdict(dict)                # define a dict for storing the variables per subgrid
    for sg_r in range(0,n):                    # iterate over of row indices for the subgrids
        for sg_c in range(0,n):                # iterate over of column indices for the subgrids
            sg_dict[sg_r][sg_c] = []           # intialize list for a new subgrid 
            for row_num in range(0,n):         # iterate over rows of a subgrid (is multiplied by n^4 later)
                for i in range(0,pow(n,3)):    # iterate over number of variables per row in a subgrid                                                          
                    # compute variable number and append to list
                    variable = int(i+(row_num*pow(n,4))+(sg_r*pow(n,5)+sg_c*pow(n,3)))      
                    # append variable list for that box inside a dict            
                    sg_dict[sg_r][sg_c].append(variable)         

                    ##### uncomment if you want to see the variables
                    #print(f'Added {variable} in list of box({sg_r},{sg_c})')

    ##### 2. Now weight the variables that represent the same value and are inside the same box 
    for sg_r in range(0,n):                                                       # select subgrid row                                                
        for sg_c in range(0,n):                                                   # select subgrid column
            sg = sg_dict[sg_r][sg_c]                                              # fetch list of subgrid's 
            for shift in range(0,pow(n,2)):                         
                for r in range(shift,pow(n,4),pow(n,2)):
                    for t in range(r,pow(n,4),pow(n,2)):
                         
                        row_num_r = math.floor(sg[r]/pow(n,4))
                        col_num_r = math.floor((sg[r]%pow(n,4))/pow(n,2))
                        row_num_t = math.floor(sg[t]/pow(n,4))
                        col_num_t = math.floor((sg[t]%pow(n,4))/pow(n,2))

                        #print(f'ref:{row_num_r},{col_num_r}')
                        #print(f'tar:{row_num_t},{col_num_t}')

                        if row_num_r != row_num_t and col_num_r != col_num_t:
                            
                            terms.append(
                                Term(
                                    c = 1,
                                    indices = [sg[r],sg[t]]   
                                )
                            )
                        #print(f'{sg[r]},{sg[t]}') 

    #####
    ##### Constraint 5: Promote to fill in values - Values already in the sudoku get a higher weighting! Can't use Term outside of function def

    for row in range(0,pow(n,2)):                    # iterate over rows
        for col in range(0,pow(n,2)):                # iterate over columns
            for val in range(1,pow(n,2)+1):          # iterate over possible integer values
                if SudokuMatrix[row,col] == val:     # if there is a value filled in, give a much lower weight
                    terms.append(                                                  
                        Term(
                            c = -5,
                            indices = [(val-1)+col*pow(n,2)+row*pow(n,4)]   
                        )
                    )
                    #print('Directly influenced by initial conditions')
                    #print(f'{(val-1)+col*pow(n,2)+row*pow(n,4)')
                elif SudokuMatrix[row,col] == 0:                
                    
                    # if no value is filled in that cell, promote the solver to fill in a weight
                    terms.append(
                        Term(
                            c = -1,
                            indices = [(val-1)+col*pow(n,2)+row*pow(n,4)]    
                        )
                    )

                    #print('Not directly influenced by initial conditions')
                    #print(f'{(val-1)+col*pow(n,2)+row*pow(n,4)') 

    return Problem(name="SudokuOptProblem", problem_type=ProblemType.pubo, terms=terms)



############################################################################################
########## Generate cost function

##### In the beginning you can run the function that defines the different Sudokus.
Sudoku = Sudoku9A

##### Create problem instance
OptimizationProblem = SudokuProblem(Sudoku)

##### Choose the solver and parameters --- uncomment if you wish to use a different one
solver = SimulatedAnnealing(workspace, timeout = 120)   
#solver = ParallelTempering(workspace, timeout = 120)
#solver = Tabu(workspace, timeout = 120)
#solver = QuantumMonteCarlo(workspace, sweeps = 2, trotter_number = 10, restarts = 72, seed = 22, beta_start = 0.1, transverse_field_start = 10, transverse_field_stop = 0.1) # QMC is not available parameter-free yet

SolverSolution = solver.optimize(OptimizationProblem)  

# Verify whether the solution is correct 
SolvedSudoku   = ReadResults(SolverSolution['configuration'], Sudoku)



Well done! You have finished this notebook on how to construct and solve a Sudoku for the Azure QIO solvers. If you're interested in learning more about Azure's QIO solvers and samples make sure to visit:
- https://github.com/microsoft/qio-samples/tree/main/samples
- https://docs.microsoft.com/en-us/learn/paths/quantum-computing-fundamentals/ 
